Cuando hacemos una solicitud, la función `AI_COMPLETE` genera una respuesta utilizando el modelo de lenguaje que hallamos elegido. 

A la función `AI_COMPLETE` podemos proporcionarle lo siguiente:

- Un prompt en formato texto para generar una respuesta del modelo.
- Una imagen y un prompt en formato texto para generar una respuesta basada en la imagen y el prompt.
- Un prompt tipo objeto que admite varias imágenes y texto.

La función `COMPLETE` admite una gran variedad de modelos. Debemos tener en cuenta que cada modelo puede tener diferentes costes. Algunos de los modelos son:

- `claude-3-5-sonnet`
- `openai-gpt-4.1`
- `llama3-8b`
- `llama3.1-8b`
- `snowflake-arctic`

Para obtener [la lista completa de los modelos soportados](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions) puede dirigirse a la documentación de Snowflake

## `AI_COMPLETE` simple

### Sintaxis

```sql
AI_COMPLETE(
    <model>, <prompt> [ , <model_parameters>, <response_format>, <show_details> ] )
```

### Habilitar **cross-region**

Las funciones LLM de Snowflake Cortex están disponibles actualmente solo en algunas regiones. Para acceder a las LLM de regiones no incluidas, vamos a utilizar el parámetro de inferencia entre regiones `CORTEX_ENABLED_CROSS_REGION`. Para ello vamos a ejecutar lo siguiente:

In [ ]:
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'ANY_REGION';

### Ejemplos

**Generar una respuesta simple**

In [ ]:
SELECT AI_COMPLETE('snowflake-arctic', '¿Qué es la Inteligencia Artificial?');

In [ ]:
SELECT AI_COMPLETE('claude-3-5-sonnet', 'Genera un script de Python que cree una sesión de snowpark');

**Generar respuestas de una columna de una tabla**

Para este ejemplo vamos a crear una tabla de nombre `restaurantes` con una columna de `nombre` y otra con `review` y a continuación vamos a generar respuestas a partir de la columna `review`.

In [ ]:
-- Crear la base de datos
CREATE DATABASE curso_ia;

-- Crear el esquema
CREATE SCHEMA seccion_2;

-- Crear la tabla
CREATE TABLE curso_ia.seccion_2.restaurantes (
    nombre VARCHAR(100),
    review VARCHAR(1000)
);

-- Insertar 10 registros
INSERT INTO curso_ia.seccion_2.restaurantes (nombre, review)
VALUES
    ('El Rincón Mediterráneo', 'Excelente comida mediterránea. Sus platos de pescado fresco son excepcionales y el servicio muy atento. Ambiente acogedor con terraza perfecta para noches de verano.'),
    ('Sabores de Asia', 'Auténtica cocina asiática con ingredientes importados. El pad thai y los dumplings son impresionantes. Porciones generosas y precios razonables.'),
    ('La Trattoria', 'La mejor pasta casera de la ciudad. Sus salsas son preparadas al momento y la carta de vinos es extensa. El tiramisú es imperdible.'),
    ('Asador El Roble', 'Carnes a la parrilla de primera calidad. El chuletón madurado 45 días vale cada centavo. Servicio un poco lento en horas pico pero la calidad compensa la espera.'),
    ('Mariscos del Puerto', 'Frescos mariscos y excelente ubicación frente al mar. La paella es espectacular aunque algo cara. Ideal para ocasiones especiales.'),
    ('Taquería Don Pancho', 'Tacos auténticos mexicanos a precios accesibles. El pastor y la cochinita pibil son deliciosos. Local pequeño pero con mucho sabor y ambiente familiar.'),
    ('Bistró Parisino', 'Encantador restaurante francés con platos clásicos bien ejecutados. El confit de pato es sublime y los postres son obras de arte. Servicio formal pero cálido.'),
    ('Veggie Garden', 'Propuesta vegetariana creativa y sabrosa. Sus hamburguesas de garbanzos y el curry de verduras son fantásticos. Personal conocedor y amable.'),
    ('Brasas & Brasas', 'Parrillada espectacular con carnes argentinas de primera. El chimichurri casero es adictivo. Ambiente rústico y acogedor perfecto para cenas en grupo.'),
    ('Sushi Sakura', 'El mejor sushi de la zona con pescado ultra fresco. El omakase del chef sorprende siempre con creaciones únicas. Precio elevado pero justificado por la calidad.')
;

In [ ]:
-- Generar respuestas a partir de la columna review

SELECT AI_COMPLETE(
    'mistral-large',
        CONCAT('Haz una crítica de estos reviews en formato de bullets: <review>', review, '</review>')
) FROM curso_ia.seccion_2.restaurantes;

### Control de temperatura y tokens

Este ejemplo ilustra el uso de los parámetros del modelo para controlar los hiperparámetros de inferencia en una sola respuesta.

`temperature`: Es un valor de 0 a 1 (inclusive) que controla la aleatoriedad de la salida del modelo de lenguaje. Una temperatura más alta (por ejemplo, 0,8) produce una salida más diversa y aleatoria, mientras que una temperatura más baja (como 0,3) la hace más determinista y específica. El valor predeterminado es 0

`max_tokens`: Establece el número máximo de tokens de salida en la respuesta. Valores bajos pueden resultar en respuestas truncadas. El valor predeterminado es 4096. El valor máximo permitido es 8192.

In [ ]:
SELECT AI_COMPLETE(
    model => 'openai-gpt-4.1',
    prompt => '¿Quién es el creador de Python?',
    model_parameters => {
		'temperature': 0.6,
        'max_tokens': 50
    }
);

### Salida detallada

El siguiente ejemplo muestra cómo usar el argumento `show_details` para obtener detalles adicionales de la inferencia.

In [ ]:
SELECT AI_COMPLETE(
    model => 'openai-gpt-4.1',
    prompt => '¿Quién es el creador de Python?',
    model_parameters => {
		'temperature': 0.6,
        'max_tokens': 50
    },
    show_details => true
);

Como se observa la respuesta es un objeto JSON que contiene el mensaje del modelo de lenguaje y otra información.

### Especificar un formato de respuesta JSON

Este ejemplo ilustra el uso del parámetro `response_format` para devolver una respuesta estructurada.

Para mostrar esto vamos a crear otra tabla muy parecida a la anterior pero esta vez con reseñas de hoteles.

In [ ]:
-- Crear la tabla
CREATE TABLE curso_ia.seccion_2.hoteles (
    nombre VARCHAR(100),
    review VARCHAR(1000)
);

-- Insertar 10 registros con reviews de diferentes sentimientos
INSERT INTO curso_ia.seccion_2.hoteles (nombre, review)
VALUES
    ('Grand Palace Hotel', 'Quedamos maravillados con la experiencia. Las habitaciones son espectacularmente espaciosas, el personal siempre disponible para cualquier necesidad y el desayuno buffet superó todas nuestras expectativas. Sin duda volveremos en nuestra próxima visita.'),
    
    ('Seaside Resort', 'La ubicación frente al mar es inmejorable. Disfrutamos cada mañana de un amanecer desde nuestro balcón privado. Las camas son tan cómodas que dormimos como bebés. El acceso directo a la playa privada fue la cereza del pastel.'),
    
    ('Mountain View Lodge', 'Las instalaciones están bastante limpias, aunque el mobiliario parece algo antiguo. La ubicación es conveniente para visitar las atracciones principales. El desayuno cumple sin sorprender. Es una opción correcta si no buscas lujos.'),
    
    ('Urban Boutique Hotel', 'La habitación era mucho más pequeña de lo que mostraban las fotos en la web. El baño necesita una renovación urgente y el aislamiento acústico es prácticamente inexistente. Pagamos demasiado para lo que recibimos.'),
    
    ('Business Inn', 'Cumplió con lo básico para un viaje de trabajo. La conexión WiFi funcionó sin problemas y la ubicación cerca del centro de convenciones fue práctica. El desayuno era variado aunque nada del otro mundo. Volvería por motivos de trabajo.'),
    
    ('Riverside Hotel', 'Nunca habíamos tenido una experiencia tan desagradable. Las sábanas estaban manchadas, encontramos polvo por todas partes y el ruido del aire acondicionado no nos dejó dormir. El personal fue completamente indiferente a nuestras quejas.'),
    
    ('Skyline Towers', 'La vista desde el piso 30 es absolutamente impresionante. El servicio de concierge resolvió todas nuestras necesidades y la piscina en la azotea es un sueño. Vale cada centavo que pagamos por la estancia.'),
    
    ('Cozy Inn', 'Es un hotel correcto para pasar un par de noches. La habitación estaba limpia aunque el baño necesita algunas mejoras. El desayuno es básico pero suficiente. La relación calidad-precio es adecuada si no esperas grandes lujos.'),
    
    ('Historic Palace Hotel', 'La ubicación es buena, pero ahí terminan los aspectos positivos. Habitación húmeda, personal desinteresado y un desayuno que dejaba mucho que desear. Por el precio que cobran, esperábamos mucho más.'),
    
    ('Garden Resort & Spa', 'Los jardines son un verdadero oasis de paz. El tratamiento en el spa fue revitalizante y las cenas en el restaurante gourmet inolvidables. El personal se aprende tu nombre desde el primer día y los detalles en la habitación son exquisitos.')
;

Ahora utilizamos la función `AI_COMPLETE` con el parámetro `response_format`.

In [ ]:
SELECT AI_COMPLETE(
    model => 'llama2-70b-chat',
    prompt => CONCAT('Del siguiente review de hotel extrae el sentimiento de las siguientes categorías: <comida,habitaciones,precio,ubicación>, en caso de no encontar información sobre alguna categoría responde con <no se menciona> para esa categoría.', review),
    model_parameters => {
        'temperature': 0,
        'max_tokens': 4096
    },
    response_format => {
            'type':'json',
            'schema':{'type' : 'object','properties' : {'sentimientos_categorias':{'type':'array','items':{'type':'object','properties':
            {'comida' : {'type' : 'string'},'habitaciones': {'type':'string'}, 'precio': {'type':'string'}, 'ubicacion': {'type':'string'}},'required':['comida','habitaciones' ,'precio','ubicacion']}}}}
    },
    show_details => true
) from curso_ia.seccion_2.hoteles;